In [ ]:
### Import some libraries
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision import transforms
from torch.autograd import Variable
from torchvision.datasets import FashionMNIST
from matplotlib import pyplot as plt
import os

In [ ]:
if not os.path.exists('./CNN_AE'):
    os.mkdir('./CNN_AE')

In [ ]:
### Define data loading step
batch_size = 128

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = FashionMNIST('./data', train = True, transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
### Define model architecture 
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16, kernel_size=3, stride=3, padding=1),#output of size 10
            nn.LeakyReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),#output of size 5
           
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, stride=2, padding=1),#output of size 3
            nn.LeakyReLU(),
            nn.AvgPool2d(kernel_size=2, stride=1, padding=0), # output size = 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=0),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(in_channels=16, out_channels=8, kernel_size=5, stride=3, padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=2, stride=2, padding=1),
            nn.Tanh(),
        )
        
       
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
model = Autoencoder().to(device)
criterion = nn.MSELoss()

In [ ]:
### Model Parameters Checking

print("\t Parameters size of our model:")
for name, param in model.named_parameters():
    print(name, param.size())
print("\n\n")

def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np
print("\t Total number of parameters in our model: ",get_n_params(model))

In [ ]:
### Configure the optimiser
learning_rate = 1e-3
l2_regularizer = 1e-5
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
    weight_decay=l2_regularizer,
)

In [ ]:
num_epochs = 15
loss_list = list()
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        #print(img.size())
        #img = Variable(img)
        
        # ===================forward=====================
        output = model(img)        
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # ===================log=========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.data[0]))
    loss_list.append(loss.data[0])
    if epoch % 1 == 0:
        pic = to_img(output.data)
        save_image(pic, './CNN_AE/{}.png'.format(epoch + 1))
print('Finished Training')


### Plot of the loss of training

nb_epoch = range(1, len(loss_list) + 1)
plt.plot(nb_epoch, loss_list,linewidth = 0.5, color = "red")
plt.xlabel('Epochs ',fontsize=9)
plt.ylabel('Loss',fontsize=9)
plt.title('Plot of the Training Loss',fontsize=16)
plt.show()